## 📋 Overview

Phase 1의 개선 버전으로, 다음과 같은 주요 개선사항을 포함합니다:

### Key Improvements
- ✅ **Multilingual Support**: Llama-3.1-8B native multilingual understanding
- ✅ **Fixed English Output**: All reports in English regardless of input language
- ✅ **Code-switching Handling**: Understands mixed-language content naturally
- ✅ **Modular Design**: 모델, 프롬프트, 전처리 모듈화
- ✅ **Extensible Architecture**: Team model 통합 준비
- ✅ **Better Preprocessing**: URL 제거, 언어 감지

### Multilingual Processing
- **Input**: Korean (한글), English, Japanese (日本語), Mixed languages
- **Processing**: LLM native understanding (no translation layer)
- **Output**: English (fixed)

### Expected Quality
- Korean input → English output: **7-9/10**
- English input → English output: **8-9/10**
- Mixed language input → English output: **7-8/10**
- Japanese input → English output: **7-8/10**

---

## 📦 1. Setup & Installation

In [ ]:
# Install required packages
!pip install -q transformers accelerate bitsandbytes torch
!pip install -q langdetect isodate
!pip install -q google-api-python-client  # For YouTube API (optional)

In [ ]:
import json
import re
import warnings
from datetime import datetime
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass, field

import torch
import isodate
from langdetect import detect, LangDetectException
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)

warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## ⚙️ 2. Configuration

모든 설정을 여기서 제어할 수 있습니다.

In [ ]:
@dataclass
class PipelineConfig:
    """전체 파이프라인 설정"""
    
    # Model Configuration
    model_name: str = "meta-llama/Llama-3.1-8B-Instruct"
    use_4bit: bool = True
    max_new_tokens: int = 512
    temperature: float = 0.7
    top_p: float = 0.9
    
    # Data Configuration
    use_youtube_api: bool = False
    youtube_api_key: Optional[str] = None
    data_path: str = "full_dataset_20251013_215347.json"
    
    # Processing Configuration
    max_description_length: int = 2000
    max_comments_to_process: int = 50
    min_comment_length: int = 10
    remove_urls: bool = True
    detect_language: bool = True  # Still detect for logging
    
    # Multilingual Configuration (NEW)
    output_language: str = "English"  # Fixed output language
    multilingual_understanding: bool = True  # LLM native multilingual
    
    # Team Model Integration
    use_category_model: bool = False
    use_sentiment_model: bool = False
    category_model_path: Optional[str] = None
    sentiment_model_path: Optional[str] = None
    
    # Output Configuration
    output_format: str = "markdown"
    save_reports: bool = True
    output_dir: str = "reports"

# Create configuration
config = PipelineConfig(
    model_name="meta-llama/Llama-3.1-8B-Instruct",
    use_4bit=True,
    use_youtube_api=False,
    data_path="full_dataset_20251013_215347.json",
    output_language="English",  # NEW: Fixed to English
    multilingual_understanding=True  # NEW: Enable multilingual
)

print("="*60)
print("Pipeline Configuration")
print("="*60)
print(f"Model: {config.model_name}")
print(f"4-bit: {config.use_4bit}")
print(f"Temperature: {config.temperature}")
print(f"Output Language: {config.output_language}")
print(f"Multilingual Understanding: {config.multilingual_understanding}")
print(f"YouTube API: {config.use_youtube_api}")
print(f"Category Model: {config.use_category_model}")
print(f"Sentiment Model: {config.use_sentiment_model}")
print("="*60)

## 🧩 3. Prompt Templates

In [ ]:
class PromptTemplates:
    """프롬프트 템플릿 관리 (Multilingual Native)"""
    
    # ===== Video Summary Prompts =====
    VIDEO_SUMMARY_SYSTEM = """You are an expert multilingual content analyst specializing in YouTube video analysis.
Your task is to understand content in ANY language (Korean, English, Japanese, or mixed) and create summaries in English.
You have native-level understanding of multiple languages and can capture nuances across different cultures.
Focus on accuracy and avoid hallucinations."""
    
    VIDEO_SUMMARY_USER = """Analyze this YouTube video and create a summary.

Video Information:
- Title: {title}
- Description: {description}
- Category: {category}
- Duration: {duration} seconds
- Channel: {channel}

Context:
The title and description may be in Korean (한글), English, Japanese (日本語), or mixed languages.
Please understand the content in its original language(s) and provide your analysis.

Instructions:
1. Read and comprehend the content regardless of the language(s) used
2. Understand cultural context and nuances in the original language
3. Write a 3-5 sentence summary in ENGLISH
4. Capture the key points, main theme, and purpose of the video
5. Be accurate - do NOT make up information or misinterpret due to language barriers
6. If description is minimal, acknowledge this limitation

Summary (in English):"""
    
    # ===== Reaction Summary Prompts =====
    REACTION_SUMMARY_SYSTEM = """You are an expert in multilingual social media sentiment analysis.
You can understand and analyze comments in Korean (한글), English, Japanese (日本語), and mixed languages.
Your task is to capture audience reactions across all language communities and summarize in English."""
    
    REACTION_SUMMARY_USER = """Analyze these YouTube comments and summarize the audience reaction.

Video: {title}

Audience Comments:
{comments}

Context:
These comments are from a multilingual audience and may include:
- Korean (한글) comments
- English comments
- Japanese (日本語) comments
- Mixed-language comments (e.g., "이 노래 진짜 beautiful하다" - Korean + English in one comment)
- Code-switching between languages

Your Task:
Please analyze ALL comments by:
1. Reading and understanding each comment in its original language(s)
2. For mixed-language comments, understanding the complete meaning and emotional tone
3. Identifying sentiment patterns (positive/negative/neutral) across all language groups
4. Finding common themes and topics that appear across different languages
5. Noting any cultural references or language-specific expressions

Instructions:
1. Comprehend ALL comments regardless of language
2. Identify overall sentiment (positive, negative, or mixed)
3. Highlight common themes that appear across language communities
4. Mention notable reactions or insightful comments
5. Write a 3-5 sentence summary in ENGLISH
6. Be objective and balanced in capturing diverse reactions
7. If different language communities show different reactions, mention this

Audience Reaction Summary (in English):"""
    
    CUSTOM_PROMPT = None
    
    @classmethod
    def get_video_summary_prompt(cls, video_info: Dict) -> List[Dict]:
        """Generate video summary prompt (language-agnostic)"""
        if cls.CUSTOM_PROMPT:
            return cls.CUSTOM_PROMPT
        
        return [
            {"role": "system", "content": cls.VIDEO_SUMMARY_SYSTEM},
            {"role": "user", "content": cls.VIDEO_SUMMARY_USER.format(
                title=video_info.get('title', 'N/A'),
                description=video_info.get('description', 'N/A')[:2000],
                category=video_info.get('category_name', 'N/A'),
                duration=video_info.get('duration_seconds', 'N/A'),
                channel=video_info.get('channel_title', 'N/A')
            )}
        ]
    
    @classmethod
    def get_reaction_summary_prompt(cls, title: str, comments: str) -> List[Dict]:
        """Generate reaction summary prompt (language-agnostic)"""
        if cls.CUSTOM_PROMPT:
            return cls.CUSTOM_PROMPT
        
        return [
            {"role": "system", "content": cls.REACTION_SUMMARY_SYSTEM},
            {"role": "user", "content": cls.REACTION_SUMMARY_USER.format(
                title=title,
                comments=comments
            )}
        ]

print("✅ Multilingual prompt templates loaded")
print("📝 Output language: English (fixed)")
print("🌍 Input languages: Korean, English, Japanese, Mixed")

### 🌍 Multilingual Processing Strategy

**Approach**: LLM Native Multilingual Understanding

**How it works:**
1. **Input**: Content in ANY language (Korean, English, Japanese, Mixed)
2. **Processing**: Llama-3.1-8B understands content in original language(s)
3. **Output**: Summary always in English

**Examples:**
```
Input:  "NMIXX(엔믹스) 'Blue Valentine' M/V"
Output: "This is NMIXX's music video for 'Blue Valentine'..."

Input:  "이 노래 진짜 좋다 I love it so much"
Output: "Positive reaction praising the song..."

Input:  "この曲最高！choreography も great"
Output: "Enthusiastic praise for the song and choreography..."
```

**Benefits:**
- ✅ No translation layer needed
- ✅ Preserves cultural context and nuances
- ✅ Handles code-switching naturally
- ✅ Fast and efficient (single LLM call)
- ✅ Consistent English output for all reports

**Language Detection:**
- Still performed for logging/debugging
- Helps monitor language distribution
- Not used for prompt selection (English output always)

---

## 🔧 4. Text Preprocessing

In [ ]:
class TextPreprocessor:
    """텍스트 전처리"""
    
    def __init__(self, config: PipelineConfig):
        self.config = config
        self.url_pattern = re.compile(r'http[s]?://\S+')
    
    def remove_urls(self, text: str) -> str:
        return self.url_pattern.sub('', text)
    
    def detect_language(self, text: str) -> str:
        try:
            lang = detect(text)
            lang_map = {'ko': 'Korean', 'en': 'English', 'ja': 'Japanese'}
            return lang_map.get(lang, 'English')
        except:
            return 'English'
    
    def clean_description(self, description: str) -> str:
        if not description:
            return "No description available."
        if self.config.remove_urls:
            description = self.remove_urls(description)
        description = ' '.join(description.split())
        if len(description) > self.config.max_description_length:
            description = description[:self.config.max_description_length] + "..."
        if len(description.strip()) < 20:
            return "Minimal description available."
        return description
    
    def filter_comments(self, comments: List[Dict]) -> List[Dict]:
        filtered = [c for c in comments if len(c.get('text', '')) >= self.config.min_comment_length]
        filtered.sort(key=lambda x: x.get('like_count', 0), reverse=True)
        return filtered[:self.config.max_comments_to_process]
    
    def format_comments_for_prompt(self, comments: List[Dict]) -> str:
        if not comments:
            return "No comments available."
        formatted = []
        for i, comment in enumerate(comments, 1):
            text = comment.get('text', '')
            likes = comment.get('like_count', 0)
            if self.config.remove_urls:
                text = self.remove_urls(text)
            formatted.append(f"{i}. [{likes} likes] {text}")
        return "\n".join(formatted)
    
    def parse_duration(self, duration_str: str) -> int:
        try:
            duration = isodate.parse_duration(duration_str)
            return int(duration.total_seconds())
        except:
            return 0

preprocessor = TextPreprocessor(config)
print("✅ Text preprocessor initialized")

## 🤖 5. Model Loading

In [ ]:
class ModelManager:
    """LLM 모델 관리"""
    
    def __init__(self, config: PipelineConfig):
        self.config = config
        self.model = None
        self.tokenizer = None
        self.pipe = None
    
    def load_model(self):
        print(f"\n🔄 Loading: {self.config.model_name}")
        print(f"⚙️ 4-bit: {self.config.use_4bit}")
        
        # Tokenizer
        print("Loading tokenizer...")
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.config.model_name, trust_remote_code=True
        )
        self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # Quantization config
        if self.config.use_4bit:
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True
            )
        else:
            quantization_config = None
        
        # Model
        print("Loading model...")
        self.model = AutoModelForCausalLM.from_pretrained(
            self.config.model_name,
            quantization_config=quantization_config,
            device_map="auto",
            trust_remote_code=True,
            torch_dtype=torch.float16 if not self.config.use_4bit else None
        )
        
        # Pipeline
        print("Creating pipeline...")
        self.pipe = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            max_new_tokens=self.config.max_new_tokens,
            temperature=self.config.temperature,
            top_p=self.config.top_p,
            do_sample=True,
            pad_token_id=self.tokenizer.eos_token_id
        )
        
        print("✅ Model loaded!")
    
    def generate(self, messages: List[Dict]) -> str:
        if self.pipe is None:
            raise RuntimeError("Model not loaded")
        outputs = self.pipe(messages)
        generated_text = outputs[0]["generated_text"]
        if isinstance(generated_text, list):
            return generated_text[-1]["content"]
        return generated_text

model_manager = ModelManager(config)
model_manager.load_model()

## 🔌 6. Team Model Integration (Optional)

In [ ]:
class TeamModelIntegration:
    """팀 모델 통합 인터페이스"""
    
    def __init__(self, config: PipelineConfig):
        self.config = config
        self.category_model = None
        self.sentiment_model = None
    
    def load_category_model(self):
        if not self.config.use_category_model:
            return
        # TODO: 팀원 구현
        print("⚠️ Category model not implemented yet")
    
    def load_sentiment_model(self):
        if not self.config.use_sentiment_model:
            return
        # TODO: 팀원 구현
        print("⚠️ Sentiment model not implemented yet")
    
    def predict_category(self, video_info: Dict) -> Optional[str]:
        if not self.config.use_category_model or self.category_model is None:
            return None
        # TODO: 모델 inference
        return None
    
    def analyze_sentiment(self, comments: List[Dict]) -> Optional[Dict]:
        if not self.config.use_sentiment_model or self.sentiment_model is None:
            return None
        # TODO: 모델 inference
        return None

team_models = TeamModelIntegration(config)
team_models.load_category_model()
team_models.load_sentiment_model()
print("✅ Team model interface ready")

## 📥 7. Data Loading

In [ ]:
class DataLoader:
    def __init__(self, config: PipelineConfig):
        self.config = config
    
    def load_from_file(self, file_path: str) -> List[Dict]:
        print(f"\n📂 Loading: {file_path}")
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"✅ Loaded {len(data)} videos")
        return data
    
    def get_data(self) -> List[Dict]:
        if self.config.use_youtube_api:
            raise NotImplementedError("YouTube API not implemented yet")
        return self.load_from_file(self.config.data_path)

data_loader = DataLoader(config)
print("✅ Data loader ready")

## 🎬 8. Report Generation Pipeline

In [ ]:
class ReportGenerator:
    def __init__(self, config, model_manager, preprocessor, team_models):
        self.config = config
        self.model_manager = model_manager
        self.preprocessor = preprocessor
        self.team_models = team_models
    
    def generate_video_summary(self, video_info: Dict) -> str:
        """Generate video summary (multilingual input → English output)"""
        # Preprocess
        video_info['description'] = self.preprocessor.clean_description(
            video_info.get('description', '')
        )
        video_info['duration_seconds'] = self.preprocessor.parse_duration(
            video_info.get('duration', 'PT0S')
        )
        
        # Detect language for logging purposes
        if self.config.detect_language:
            detected_lang = self.preprocessor.detect_language(
                video_info.get('title', '') + ' ' + video_info.get('description', '')[:500]
            )
            print(f"    [Detected input language: {detected_lang}]")
        
        # Generate prompt (no language parameter - LLM handles multilingual)
        messages = PromptTemplates.get_video_summary_prompt(video_info)
        
        # Generate summary
        try:
            summary = self.model_manager.generate(messages)
            print(f"    [Output language: {self.config.output_language}]")
            return summary.strip()
        except Exception as e:
            print(f"⚠️ Error: {e}")
            return "Summary generation failed."
    
    def generate_reaction_summary(self, title: str, comments: List[Dict]) -> str:
        """Generate reaction summary (multilingual input → English output)"""
        # Filter and format comments
        filtered_comments = self.preprocessor.filter_comments(comments)
        
        if not filtered_comments:
            return "No comments available."
        
        comments_text = self.preprocessor.format_comments_for_prompt(filtered_comments)
        
        # Detect language distribution for logging
        if self.config.detect_language:
            sample_texts = [c.get('text', '')[:100] for c in filtered_comments[:10]]
            langs = [self.preprocessor.detect_language(t) for t in sample_texts if t]
            lang_dist = {}
            for lang in langs:
                lang_dist[lang] = lang_dist.get(lang, 0) + 1
            print(f"    [Comment languages: {lang_dist}]")
        
        # Generate prompt (no language parameter)
        messages = PromptTemplates.get_reaction_summary_prompt(title, comments_text)
        
        # Generate summary
        try:
            summary = self.model_manager.generate(messages)
            print(f"    [Output language: {self.config.output_language}]")
            return summary.strip()
        except Exception as e:
            print(f"⚠️ Error: {e}")
            return "Reaction summary generation failed."
    
    def calculate_engagement_metrics(self, video_info: Dict) -> Dict:
        views = video_info.get('view_count', 0)
        likes = video_info.get('like_count', 0)
        comments = video_info.get('comment_count', 0)
        
        engagement_rate = ((likes + comments) / views * 100) if views > 0 else 0
        like_rate = (likes / views * 100) if views > 0 else 0
        comment_rate = (comments / views * 100) if views > 0 else 0
        
        return {
            'views': views,
            'likes': likes,
            'comments': comments,
            'engagement_rate': round(engagement_rate, 2),
            'like_rate': round(like_rate, 2),
            'comment_rate': round(comment_rate, 2)
        }

print("✅ Report generator class defined (multilingual)")

In [ ]:
def format_markdown_report(self, video_data: Dict, video_summary: str, 
                           reaction_summary: str, metrics: Dict,
                           team_predictions: Dict = None) -> str:
    video_info = video_data['video_info']
    comments = video_data.get('comments', [])
    
    report = f"""# YouTube Video Report

**Generated**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Model**: {self.config.model_name}

---

## 📹 Video Information

- **Title**: {video_info.get('title', 'N/A')}
- **Channel**: {video_info.get('channel_title', 'N/A')}
- **Category**: {video_info.get('category_name', 'N/A')}
- **Published**: {video_info.get('published_at', 'N/A')}
- **Duration**: {self.preprocessor.parse_duration(video_info.get('duration', 'PT0S'))} seconds
- **Video ID**: `{video_info.get('video_id', 'N/A')}`
- **URL**: https://www.youtube.com/watch?v={video_info.get('video_id', '')}

---

## 📊 Engagement Metrics

| Metric | Value |
|--------|-------|
| Views | {metrics['views']:,} |
| Likes | {metrics['likes']:,} |
| Comments | {metrics['comments']:,} |
| Engagement Rate | {metrics['engagement_rate']}% |
| Like Rate | {metrics['like_rate']}% |
| Comment Rate | {metrics['comment_rate']}% |

---

## 📝 Video Summary

{video_summary}

---

## 💬 Audience Reaction Summary

{reaction_summary}

---
"""
    
    if team_predictions:
        report += """## 🤖 Team Model Predictions\n\n"""
        if 'category' in team_predictions:
            report += f"- **Category**: {team_predictions['category']}\n"
        if 'sentiment' in team_predictions:
            s = team_predictions['sentiment']
            report += f"- **Sentiment**: Positive {s.get('positive',0):.0%}, "
            report += f"Neutral {s.get('neutral',0):.0%}, Negative {s.get('negative',0):.0%}\n"
        report += "\n---\n\n"
    
    top_comments = self.preprocessor.filter_comments(comments)[:5]
    if top_comments:
        report += """## 🔍 Top Comments\n\n"""
        for i, c in enumerate(top_comments, 1):
            text = c.get('text', '')[:200]
            likes = c.get('like_count', 0)
            author = c.get('author', 'Anonymous')
            report += f"{i}. **{author}** ({likes} likes): {text}...\n\n"
        report += "---\n\n"
    
    report += """*Generated by YouTube Report Generator - Phase 2*"""
    return report

# Add method to ReportGenerator class
ReportGenerator.format_markdown_report = format_markdown_report
print("✅ Report formatting method added")

In [ ]:
def generate_report(self, video_data: Dict) -> str:
    video_info = video_data['video_info']
    comments = video_data.get('comments', [])
    
    print(f"\n🎬 Processing: {video_info.get('title', 'Unknown')}")
    
    print("  📝 Generating video summary...")
    video_summary = self.generate_video_summary(video_info)
    
    print("  💬 Generating reaction summary...")
    reaction_summary = self.generate_reaction_summary(
        video_info.get('title', ''), comments
    )
    
    print("  📊 Calculating metrics...")
    metrics = self.calculate_engagement_metrics(video_info)
    
    team_predictions = {}
    if self.config.use_category_model:
        pred = self.team_models.predict_category(video_info)
        if pred:
            team_predictions['category'] = pred
    
    if self.config.use_sentiment_model:
        sent = self.team_models.analyze_sentiment(comments)
        if sent:
            team_predictions['sentiment'] = sent
    
    print("  📄 Formatting report...")
    report = self.format_markdown_report(
        video_data, video_summary, reaction_summary, 
        metrics, team_predictions if team_predictions else None
    )
    
    print("  ✅ Done!")
    return report

# Add method
ReportGenerator.generate_report = generate_report

# Initialize report generator
report_generator = ReportGenerator(config, model_manager, preprocessor, team_models)
print("✅ Report generator initialized")

## 🚀 9. Run Pipeline

In [ ]:
# Load data
dataset = data_loader.get_data()

# Select videos (start with 3 for testing)
videos_to_process = dataset[:3]  # Change to dataset for all 20

print(f"\n🎯 Processing {len(videos_to_process)} videos...")
print("="*60)

In [ ]:
# Generate reports
reports = []

for i, video_data in enumerate(videos_to_process, 1):
    print(f"\n{'='*60}")
    print(f"Video {i}/{len(videos_to_process)}")
    print(f"{'='*60}")
    
    try:
        report = report_generator.generate_report(video_data)
        reports.append({
            'video_id': video_data['video_info']['video_id'],
            'report': report
        })
    except Exception as e:
        print(f"❌ Error: {e}")
        continue

print(f"\n{'='*60}")
print(f"✅ Completed {len(reports)}/{len(videos_to_process)} videos")
print(f"{'='*60}")

## 💾 10. Save Reports

In [ ]:
import os

if config.save_reports:
    os.makedirs(config.output_dir, exist_ok=True)
    
    print(f"\n💾 Saving to: {config.output_dir}/")
    
    for report_data in reports:
        video_id = report_data['video_id']
        report = report_data['report']
        
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"{config.output_dir}/report_{video_id}_{timestamp}.md"
        
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(report)
        
        print(f"  ✅ {filename}")
    
    print(f"\n✅ All reports saved!")

## 📊 11. Display Sample Report

In [ ]:
if reports:
    print("\n" + "="*60)
    print("Sample Report (First Video)")
    print("="*60 + "\n")
    print(reports[0]['report'])
else:
    print("⚠️ No reports generated")

## 🎓 12. Usage Tips

### 🌍 Multilingual Processing

**The pipeline automatically handles:**
```python
# Korean video → English report
Input:  Title: "NMIXX(엔믹스) 'Blue Valentine' M/V"
        Comments: "이 노래 진짜 좋다", "완전 대박"
Output: English summary capturing all Korean content

# Mixed language → English report  
Input:  Comments: "이 노래 beautiful하다", "choreography 진짜 amazing"
Output: English summary understanding code-switching

# Japanese → English report
Input:  Comments: "この曲最高！", "lyrics も素晴らしい"
Output: English summary from Japanese reactions
```

**Configuration:**
```python
config = PipelineConfig(
    output_language="English",           # Fixed output
    multilingual_understanding=True,     # Enable LLM native multilingual
    detect_language=True                 # For logging only
)
```

### 🔧 Change Model
```python
config = PipelineConfig(
    model_name="google/gemma-2-9b-it"  # or other models
)
```

### 📝 Customize Prompts
```python
# Modify multilingual instructions
PromptTemplates.VIDEO_SUMMARY_USER = """Your custom prompt..."""

# Add more languages
PromptTemplates.VIDEO_SUMMARY_SYSTEM = """
You understand Korean, English, Japanese, Chinese, Spanish...
"""
```

### 🤖 Add Team Models
```python
config = PipelineConfig(
    use_category_model=True,
    category_model_path="path/to/model"
)
```

### 📊 Process All Videos
```python
videos_to_process = dataset  # All 20 videos
```

### 🐛 Debugging Multilingual Issues

**Check language detection logs:**
```
🎬 Processing: NMIXX(엔믹스) "Blue Valentine" M/V
  📝 Generating video summary...
    [Detected input language: Korean]    ← Input language
    [Output language: English]           ← Output fixed
  💬 Generating reaction summary...
    [Comment languages: {'English': 7, 'Korean': 3}]  ← Language mix
    [Output language: English]
```

**If output quality is poor:**
1. Check if input language is too rare (e.g., not Korean/English/Japanese)
2. Try adjusting temperature (lower = more deterministic)
3. Add language-specific examples to prompt
4. Consider Option 3 (pre-translation) for very rare languages

---

**Expected Performance**:
- Korean → English quality: **7-9/10** (vs 0-2/10 Phase 1)
- English → English quality: **8-9/10** (vs 4-8/10 Phase 1)
- Mixed language quality: **7-8/10** (NEW)
- Time: ~2-3 min/video on T4 GPU

**Next Steps**:
1. Test with 3 sample videos (different languages)
2. Verify all outputs are in English
3. Check if multilingual content is understood correctly
4. Compare with Phase 1 quality
5. Try different models if needed
6. Process full dataset

---

✅ **Phase 2 Full Pipeline with Multilingual Support Ready!**